# Szyfry strumieniowe

Jedną z najpopularniejszych metod realizacji generatorów klucza jest wykorzystanie szyfrów blokowych. Ze względu na swoje właściwości (możliwość rozpoczęcia generacji od dowolnego miejsca) bardzo chętnie stosuje licznikowy tryb pracy szyfru w którym szyfruje się kolejne wartości licznika. W tym trybie szyfr blokowy zawsze pracuje jako algorytm szyfrujący, a połączone kolejne szyfrogramy stanowią wyjściowe szyfrogramy. Ponieważ kolejne wartości licznika są szyfrowane niezależnie, więc można powiedzieć, że szyfr blokowy pracuje w trybie ECB. Właściwe szyfrowanie odbywa się poprzez łączenie operacją xor tekstu jawnego ze strumieniem klucza. Oczywiście generator klucza po stronie odbiorczej musi być zainicjowany tak samo jak generator po stronie nadawczej. Dzięki przekształceniu szyfru blokowego w strumieniowy unika się konieczności dopełniania danych.

## AKA

Przed wykonaniem szyfrowania i deszyfrowania strony muszą być wyposażone w klucz tajny. Jeżeli współdzielona tajemnica jest tylko hasłem o niskiej entropii to należy przeprowadzić procedurę *key streching*. Oprócz współdzielonego hasła jej parametrem jest współdzielona i jawna liczba losowa *salt*. Procedury te zostaną omówione przy innej okazji. Tutaj zostanie przedstawiona tylko procedura która realizuje to zadanie. Procedurę tę muszą wykonać nadawca i odbiorca.

In [3]:
import binascii
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
# Key stretching
password = b'password123'
salt = get_random_bytes(16)
secret_key = PBKDF2(password, salt, 32, count=100000, hmac_hash_module=SHA256)
print("Salt= ", binascii.hexlify(salt))
print("Secret Key= ", binascii.hexlify(secret_key))

Salt=  b'c40764ece5d4745604d4f36949bab060'
Secret Key=  b'323d17590adddac6dd6bd5724bfb9ec23317507a3bf06eb0c4a81a5095f53802'


## Szyfrowanie

In [4]:
import os, binascii
from Crypto.Cipher import AES
# Prepare cipher
cipher = AES.new(secret_key, AES.MODE_CTR)
# Nonce is automatically generated
anonce = cipher.nonce
# Prepare message
message = 'To jest text do zaszyfrowania'
plaintext = message.encode('utf-8')
print("Plaintext= ", binascii.hexlify(plaintext))
ciphertext = cipher.encrypt(plaintext)
# Send encrypted data
print("Recipient receives:")
print("Nonce= ", binascii.hexlify(anonce), len(anonce))
print("Ciphertext= ", binascii.hexlify(ciphertext))
message = anonce + ciphertext

Plaintext=  b'546f206a657374207465787420646f207a61737a7966726f77616e6961'
Recipient receives:
Nonce=  b'46fe5ae542d24232' 8
Ciphertext=  b'4f8bf974785a748d3b8847e26948fa751562bb948ae61cb643ff6edf00'


## Deszyfrowanie

Po stronie odbiorczej wystarczy identycznie zainicjować generator klucza.

In [5]:
anonce = message[:8]
ciphertext = message[8:]
decryptor = AES.new( secret_key, AES.MODE_CTR, nonce=anonce)
plaintext = decryptor.decrypt(ciphertext)
print("Message= ", plaintext.decode('utf-8'))  

Message=  To jest text do zaszyfrowania


**Pamiętaj!** Samo szyfrowanie nie zabezpiecza przed manipulowaniem danymi. Należy użyć kryptograficznych sum uwierzytelniających (MAC) aby się przed tym zabezpieczyć.